In [ ]:
import os
import sys
from time import sleep

import GCode
import GRBL
import numpy as np

# Draw a Brick Pattern

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

Attempt at programmatically making a brick pattern. 

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)



In [9]:
BrickHeight = 65
BrickLength = 225
BrickDepth = 102.5
BrickRatio = 215 / 65


In [10]:
BlockHeight = 89.0
BlockLength = 176.0


In [22]:
N_BrickRows = 5
H_Block_Brick = BlockHeight / N_BrickRows
L_Block_Brick = np.round(H_Block_Brick * BrickRatio, 4)


# Code:

In [13]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()


In [14]:
home_timeout = 360

cnc.home()
for _ in range(home_timeout):
    if len(cnc.cmd("")) == 2:
        break
    sleep(1)


In [19]:
print(cnc.laser_mode)


1.0


In [20]:
cnc.cmd("G28")


['ok', 'ok']

In [21]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=200)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0, Z=0)
    return program


In [17]:
np.arange(H_Block_Brick, BlockHeight, H_Block_Brick)


array([17.8, 35.6, 53.4, 71.2])

In [25]:
Brick_Horizontal_Line = np.array(
    [[0, H_Block_Brick], [BlockLength, H_Block_Brick]]
)
Brick_Horizontal_Line


array([[  0. ,  17.8],
       [176. ,  17.8]])

In [27]:
Brick_Line_V = np.array([[L_Block_Brick, 0], [L_Block_Brick, H_Block_Brick]])
Brick_Line_V


array([[58.8769,  0.    ],
       [58.8769, 17.8   ]])

In [29]:
for row_idx in range(Brick_Line_V.shape[0]):
    print(Brick_Line_V[:, row_idx])
    break


[58.8769 58.8769]


In [31]:
default_line = np.array([[0, 0], [10, 5]])
prog = GCode.GCode()


def line(points=default_line, feed=300, power=150, dynamic_power=True):
    prog.G0(X=points[0, 0], Y=points[0, 1])

    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M4(S=power)

    for row_idx in range(1, points.shape[0]):
        prog.G1(X=points[row_idx, 0], Y=points[row_idx, 1], F=feed)
    prog.M5()


line()
prog


<GCode>[cmds=4]

In [32]:
for r in range(N_BrickRows):
    print(H_Block_Brick * r)


0.0
17.8
35.6
53.400000000000006
71.2


Horizontal Lines

In [33]:
np.arange(H_Block_Brick, BlockHeight, H_Block_Brick)


array([17.8, 35.6, 53.4, 71.2])

Odd Row Vertical Lines

In [40]:
np.arange(L_Block_Brick, BlockLength, L_Block_Brick)


array([ 58.8769, 117.7538])

Even Row Vertical Lines

In [41]:
np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick)


array([ 29.43845,  88.31535, 147.19225])

In [44]:
horizontal_lines = np.arange(H_Block_Brick, BlockHeight, H_Block_Brick)


In [45]:
horizontal_lines


array([17.8, 35.6, 53.4, 71.2])

In [48]:
np.arange(0, BlockHeight + 1, H_Block_Brick)


array([ 0. , 17.8, 35.6, 53.4, 71.2, 89. ])

In [50]:
Y = np.arange(0, BlockHeight + 1, H_Block_Brick)


In [54]:
for idx in range(len(Y) - 1):
    break


In [56]:
Y[idx]


0.0

In [58]:
Y[idx + 1]


17.8

In [66]:
np.linspace(0, BlockHeight, N_BrickRows, endpoint=False)


array([ 0. , 17.8, 35.6, 53.4, 71.2])

In [69]:
horizontal_brick_lines = np.round(
    np.linspace(0, BlockHeight, N_BrickRows, endpoint=False), 4
)
horizontal_brick_lines


array([ 0. , 17.8, 35.6, 53.4, 71.2])

In [ ]:
np.

In [78]:
flip = np.array([[1, 1], [1, 0]])
for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )

    # if np.mod(idx, 2) == 0:
    #    row_line_points = np.matmul(flip, row_line_points)
    # print(row_line_points)

    # Vertical brick line.
    if np.mod(idx, 2) == 0:
        vertical_brick_lines = np.arange(
            L_Block_Brick, BlockLength, L_Block_Brick
        )
    else:
        vertical_brick_lines = np.arange(
            L_Block_Brick / 2, BlockLength, L_Block_Brick
        )

    print(vertical_brick_lines)


[ 29.43845  88.31535 147.19225]
[ 58.8769 117.7538]
[ 29.43845  88.31535 147.19225]
[ 58.8769 117.7538]


In [79]:
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),  # Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick),  # Even rows.
)


In [87]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    flip,  # Flip the matrix, reduces travel time.
)
for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )

    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]

    row_line_points = np.matmul(transform, row_line_points)
    # print(row_line_points)

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, 2)]
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]

        column_line_points = np.array(
            [
                [start_point_y, vertical_brick_line],
                [end_point_y, vertical_brick_line],
            ]
        )
        column_line_points = np.matmul(transform, column_line_points)
        print(column_line_points)
        print()
    break


[[ 0.      29.43845]
 [17.8     29.43845]]

[[ 17.8     176.6307 ]
 [  0.       88.31535]]

[[  0.      147.19225]
 [ 17.8     147.19225]]



In [92]:
prog = GCode.GCode()
default_feed = 300
default_power = 150


def line(
    points=default_line,
    feed=default_feed,
    power=default_power,
    dynamic_power=True,
):
    prog.G0(X=points[0, 0], Y=points[0, 1])

    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M4(S=power)

    for row_idx in range(1, points.shape[0]):
        prog.G1(X=points[row_idx, 0], Y=points[row_idx, 1], F=feed)
    prog.M5()


line()
prog


<GCode>[cmds=4]

In [95]:
prog = GCode.GCode()
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    flip,  # Flip the matrix, reduces travel time.
)
for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )

    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]
    row_line_points = np.matmul(transform, row_line_points)
    line(row_line_points)

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, 2)]
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]

        column_line_points = np.array(
            [
                [start_point_y, vertical_brick_line],
                [end_point_y, vertical_brick_line],
            ]
        )
        column_line_points = np.matmul(transform, column_line_points)
        line(column_line_points)
        break

prog


<GCode>[cmds=32]